<a href="https://colab.research.google.com/github/shikhar2402/MACHINE-LEARNING-FOR-BEGINNERS/blob/master/Pytorch2PytorchLightning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import torch
from torch import nn,optim

In [7]:
from torchvision import datasets as dts
from torchvision.transforms import ToTensor 

traindt = dts.MNIST(
    root = 'data',
    train = True,                         
    transform = ToTensor(), 
    download = True,            
)
testdt = dts.MNIST(
    root = 'data', 
    train = False, 
    transform = ToTensor()
)

train_loader = torch.utils.data.DataLoader(traindt, batch_size=32, shuffle=True)

testldr = torch.utils.data.DataLoader(testdt, batch_size=32, shuffle=True)


In [34]:
model = nn.Sequential(
    nn.Linear(28*28 , 14*14),
    nn.Linear(14*14 , 64),
    nn.ReLU6(),
    nn.Linear(64 , 10)
)

optimizer = optim.RMSprop(model.parameters())

loss = nn.CrossEntropyLoss()

epochs = 5
for i in range(epochs):
  losses = []
  for batch in train_loader:
    x , y = batch
    b = x.size(0)
    x = x.view(b , -1)
    
    y_hat = model(x)
    J = loss(y_hat ,y  )

    model.zero_grad()

    J.backward()
    
    losses.append(J.item())

    optimizer.step()
  print(f'Epoch {i+1} Losses {torch.tensor(losses).mean():.2f}')


Epoch 1 Losses 2.71
Epoch 2 Losses 2.63
Epoch 3 Losses 2.63
Epoch 4 Losses 2.64
Epoch 5 Losses 2.63



**Pytorch Lightning**

In [1]:
!pip install pytorch-lightning
import pytorch_lightning as pl
print(pl.__version__)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 585 kB 5.0 MB/s 
     |████████████████████████████████| 419 kB 52.2 MB/s 
     |████████████████████████████████| 596 kB 52.5 MB/s 
     |████████████████████████████████| 140 kB 72.1 MB/s 
     |████████████████████████████████| 1.1 MB 50.2 MB/s 
     |████████████████████████████████| 94 kB 4.3 MB/s 
     |████████████████████████████████| 144 kB 73.9 MB/s 
     |████████████████████████████████| 271 kB 71.4 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
1.6.5


In [4]:
import pytorch_lightning as pl
from torchmetrics.functional import accuracy
import torch
from torch import nn,optim
from torchvision import datasets as dts
from torchvision.transforms import ToTensor 

class resnet(pl.LightningModule):
  def __init__(self):
    super().__init__()
    self.l1 = nn.Linear(28*28 , 64)
    self.l2 = nn.Linear(64 , 64)
    self.l3 = nn.Linear(64 , 10)
    self.do = nn.Dropout(0.1)
    self.loss = nn.CrossEntropyLoss()

 
  def forward(self , x):
    h1 = nn.functional.relu(self.l1(x))
    h2 = nn.functional.relu(self.l2(h1))
    do = self.do(h1 + h2)
    logits = self.l3(do)
    return logits

  def configure_optimizers(self):
    optimizer = optim.RMSprop(self.parameters())
    return optimizer
    
  def training_step(self, batch, batch_idx):
    x , y = batch
    b = x.size(0)
    x = x.view(b , -1)
    
    y_hat = self(x)

    J = self.loss(y_hat ,y )
    acc = accuracy(y_hat ,y)
    pbar = {"training_acc" : acc}
    return {"loss" : J , "avg_val_acc" : pbar}

  def train_dataloader(self):

    traindt = dts.MNIST(root = 'data', train = True, transform = ToTensor(), download = True)
    # testdt = dts.MNIST(root = 'data', train = False,  transform = ToTensor() )

    train_loader = torch.utils.data.DataLoader(traindt, batch_size=32, shuffle=True)

    # testldr = torch.utils.data.DataLoader(testdt, batch_size=32, shuffle=True)

    return train_loader

model = resnet()


Training on gpu

In [5]:
import time
start = time.time()
trainer = pl.Trainer(progress_bar_refresh_rate=20 , max_epochs=5 ,gpus=1)
trainer.fit(model)
end = time.time()
print(end - start)


/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/callback_connector.py:97: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=20)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass `enable_progress_bar = False` to the Trainer.
  f"Setting `Trainer(progress_bar_refresh_rate={progress_bar_refresh_rate})` is deprecated in v1.5 and"
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: /content/lightning_logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type             | Params
------------------------------------------
0 | l1   | Linear           | 50.2 K
1 | l2   | Linear           | 4.2 K 
2 | l3   | Linear           | 650   
3 | do   | Dro

Training: 0it [00:00, ?it/s]

72.4405882358551


Training with out gpu

In [9]:
import time
start = time.time()
trainer = pl.Trainer(progress_bar_refresh_rate=20 , max_epochs=5)
trainer.fit(model)
end = time.time()
print(end - start)


/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/callback_connector.py:97: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=20)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass `enable_progress_bar = False` to the Trainer.
  f"Setting `Trainer(progress_bar_refresh_rate={progress_bar_refresh_rate})` is deprecated in v1.5 and"
GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:1817: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  category=PossibleUserWarning,

  | Name | Type             | Params
--------------

Training: 0it [00:00, ?it/s]

59.039745807647705


In [12]:
!ls lightning_logs/version_2/checkpoints

'epoch=4-step=9375.ckpt'
